In [1]:
# from __init__ import * 
from snmcseq_utils import create_logger
from snmcseq_utils import get_cluster_mc_c
# from CEMBA_update_mysql import connect_sql
# from CEMBA_init_ensemble_v2 import pull_genebody_info
# from CEMBA_init_ensemble_v2 import pull_binc_info

In [2]:
log = create_logger()

In [3]:
ens = 'Ens1'
context = 'CH'
df = get_cluster_mc_c(ens, context, genome_regions='genebody', cluster_type='mCHmCG_lv_npc50_k30', database='CEMBA')

/cndd/fangming/venvs/venv_banjo_snmcseq/lib/python3.4/site-packages/matplotlib/__init__.py:1405: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)
/cndd/fangming/venvs/venv_banjo_snmcseq/lib/python3.4/site-packages/matplotlib/__init__.py:1405: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)
03/01/2018 09:47:41 AM Pulling genebody information (1202 cells)...
03/01/2018 09:47:41 AM Loaded cell: 171213_CEMBA_mm_P56_P63_3C_MOp_CEMBA171206_3C_4_CEMBA171206_3C_5_A11_AD002_indexed (1/1202)
03/01/2018 09:47:41 AM Loaded cell: 171213_CEMBA_mm_P56_P63_3C_MOp_CEMBA171206_3C_1_CEMBA17120

ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.